In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from TModel.TransformerModel import TranscriptionTransformerModel
from Tokenizer.loaderH5 import H5GuitarTokenizer
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
from TUtils import random_string
from lightning import Trainer
import torch
from TranscriptionDataset import TranscriptionDataset
from TModel.Retnet.TranscriptionModel import TranscriptionRetnetModel
torch.set_float32_matmul_precision('medium')

In [2]:
datasetLocation = "Trainsets/S_Tier_1695619803_mTokens400_mNoS5.hdf5"
wandbProject = "TranscriptionModel_Test"
batchSize = 8
num_workers = 2

In [3]:
dataset,pipe = TranscriptionDataset.getDataPipe(
    datasetLocation,
    batchSize,
    batchFirst=True
)
# train_pipe,test_pipe = pipe.random_split({"train":0.8,"test":0.2},42)

train_dl = DataLoader(dataset=pipe,batch_size=None,num_workers=num_workers)
# test_dl = DataLoader(dataset=test_pipe, batch_size=batchSize,num_workers=num_workers)

In [4]:
model = TranscriptionRetnetModel(
    dataset.getVocabSize(),
    d_model=512,
    d_ff=2048,
    # embeddingCheckpoint="Models/GuitarToken/Max2Length.ckpt"
)
# torch.compile(model)

In [5]:
wandb_logger = WandbLogger(project=wandbProject)
wandb_logger.experiment.config.update(dataset.meta_data)
wandb_logger.experiment.config["batchSize"] = batchSize
checkpoint_callback = ModelCheckpoint(
    monitor='train_loss',
    dirpath=f'Models/GuitarTranscription/{random_string(10)}/',
    filename='GuitarTranscriptionModel-{epoch:02d}-{train_loss:.2f}',
    every_n_train_steps=1000,
    save_top_k=3,
    mode='min',
)
trainer = Trainer(
    default_root_dir='Models/',
    max_epochs=5,
    # profiler="simple",
    # profiler="pytorch",
    logger=wandb_logger,
    callbacks=[checkpoint_callback],
    # max_time="00:00:02:00"
    # precision="bf16-mixed",    
)

wandb: Currently logged in as: draguve. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model=model, train_dataloaders=train_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                | Params
------------------------------------------------------
0 | encoder       | RetnetEncoderLayers | 20.5 M
1 | decoder       | RetnetDecoderLayers | 28.4 M
2 | tgt_embedding | Embedding           | 3.1 M 
3 | outputLinear  | Linear              | 3.1 M 
4 | loss          | CrossEntropyLoss    | 0     
------------------------------------------------------
55.2 M    Trainable params
0         Non-trainable params
55.2 M    Total params
220.606   Total estimated model params size (MB)
C:\Users\ritwi\anaconda3\envs\pytorch\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.01 GiB (GPU 0; 8.00 GiB total capacity; 11.04 GiB already allocated; 0 bytes free; 12.81 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.save_checkpoint("Models/GuitarTranscription/5s400Tokens_1e-5/smallDataset5epoch.ckpt")